# **LP3_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits. """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Let's denote the number of units produced (in-house or subcontracted) of product Pi as Xi where i [P1, P2, P3]. Also, we have decision variables: Xi_S to denote the number of units of product Pi required for smelting at PRODA and Xi_sub for subcontracted smelting units of product Pi where i [P1, P2]. 

Parameters: 

* Cp[i][j] : Cost per unit of operation j on product Pi [i: P1, P2, P3, j: smelting, mechanisation, packaging]

* Sp[i] : Sales price per unit of product Pi [i: P1, P2, P3] 

* Rp[i][j] : Required time per unit of operation j on product Pi [i: P1, P2, P3, j: smelting, mechanisation, packaging]

* Wc[j] : Weekly capacity of operation j [j: smelting, mechanisation, packaging]

Objective Function:
Maximize Profit = Σ [Sp[i]*(Xi) - Cp[i][smelting]*Xi_S - Cp[i][subcontracted]*Xi_sub - Cp[i][mechanisation]*Xi - Cp[i][packaging]*Xi] for all i 

Constraints:

1. Xi = Xi_S + Xi_sub for all i [P1,P2]

2. Σ (Xi_S*Rp[i][j]) <= Wc[j] for all j [smelting, mechanisation, packaging]

3. Σ 

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
Markdown(response2.choices[0].message.content)

Here is a sample piece of pyomo code for the problem specified:

```python
# Importing necessary libraries
from pyomo.environ import ConcreteModel, Var, NonNegativeReals, Objective, maximize, Constraint, SolverFactory, range

# Creating the model
model = ConcreteModel()

# I am using sample data for product range, cost, sales price, operation time and weekly capacity.
I = ['P1', 'P2', 'P3'] # Product range
J = ['smelting', 'mechanisation', 'packaging'] # Operation range

Cp  = {('P1','smelting'):2, ('P1','mechanisation'):3, ('P1','packaging'):3,
       ('P2','smelting'):6, ('P2','mechanisation'):4, ('P2','packaging'):3,
       ('P3','smelting'):2, ('P3','mechanisation'):2, ('P3','packaging'):4} # Cost per unit

Sp  = {'P1':10, 'P2':16, 'P3':12} # Sales price per unit

Rp  = {('P1','smelting'):1, ('P1','mechanisation'):1, ('P1','packaging'):1,
       ('P2','smelting'):1, ('P2','mechanisation'):1, ('P2','packaging'):1,
       ('P3','smelting'):1, ('P3','mechanisation'):1, ('P3','packaging'):1} # Operation time per unit

Wc  = {'smelting':15, 'mechanisation':15, 'packaging':15} # Weekly capacity of operation

# Creating the decision variables
model.X = Var(I, domain=NonNegativeReals)
model.X_S = Var(I, domain=NonNegativeReals)
model.X_sub = Var(I[:-1], domain=NonNegativeReals)

# Defining the objective function
profit = sum((Sp[i]*model.X[i] - Cp[(i,'smelting')]*model.X_S[i] - Cp[(i,'subcontracted')]*model.X_sub[i] - Cp[(i,'mechanisation')]*model.X[i] - Cp[(i,'packaging')]*model.X[i]) for i in I[:-1]) + sum((Sp[i]*model.X[i] - Cp[(i,'smelting')]*model.X[i] - Cp[(i,'mechanisation')]*model.X[i] - Cp[(i,'packaging')]*model.X[i]) for i in I[-1:]) 
model.profit = Objective(expr = profit, sense=maximize)

# Adding the constraints
model.constraints = ConstraintList()
for i in I[:-1]:
    model.constraints.add(model.X[i] == model.X_S[i] + model.X_sub[i])
    
for j in J:
    model.constraints.add(sum(model.X_S[i]*Rp[(i,j)] for i in I[:-1]) <= Wc[j])
    model.constraints.add(sum(model.X[i]*Rp[(i,j)] for i in I[-1:]) <= Wc[j])

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# Displaying the result
model.display()
```

Please replace the cost and other parameter values with real data and run the model on the installed solvers like 'glpk' or 'cplex'.

### **Run the code Generated by GPT4**

In [12]:
from pyomo.environ import ConcreteModel, Var, NonNegativeReals, Objective, maximize, Constraint, SolverFactory, range

# Creating the model
model = ConcreteModel()

# START - correct data inputted by human.
I = ['P1', 'P2', 'P3'] # Product range
J = ['smelting', 'mechanisation', 'packaging'] # Operation range

Cp = {
    'P1': {'smelting': 0.30, 'mechanisation': 0.20, 'packaging': 0.30},
    'P2': {'smelting': 0.50, 'mechanisation': 0.10, 'packaging': 0.20},
    'P3': {'smelting': 0.40, 'mechanisation': 0.27, 'packaging': 0.20}
}

Sp = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}

Rp = {
    'P1': {'smelting': 6, 'mechanisation': 6, 'packaging': 3},
    'P2': {'smelting': 10, 'mechanisation': 3, 'packaging': 2},
    'P3': {'smelting': 8, 'mechanisation': 8, 'packaging': 2}
}

Wc = {'smelting': 8000, 'mechanisation': 12000, 'packaging': 10000}
#END
# Creating the decision variables
model.X = Var(I, domain=NonNegativeReals)
model.X_S = Var(I, domain=NonNegativeReals)
model.X_sub = Var(I[:-1], domain=NonNegativeReals)

# Defining the objective function
profit = sum((Sp[i]*model.X[i] - Cp[(i,'smelting')]*model.X_S[i] - Cp[(i,'subcontracted')]*model.X_sub[i] - Cp[(i,'mechanisation')]*model.X[i] - Cp[(i,'packaging')]*model.X[i]) for i in I[:-1]) + sum((Sp[i]*model.X[i] - Cp[(i,'smelting')]*model.X[i] - Cp[(i,'mechanisation')]*model.X[i] - Cp[(i,'packaging')]*model.X[i]) for i in I[-1:])
model.profit = Objective(expr = profit, sense=maximize)

# Adding the constraints
model.constraints = ConstraintList()
for i in I[:-1]:
    model.constraints.add(model.X[i] == model.X_S[i] + model.X_sub[i])

for j in J:
    model.constraints.add(sum(model.X_S[i]*Rp[(i,j)] for i in I[:-1]) <= Wc[j])
    model.constraints.add(sum(model.X[i]*Rp[(i,j)] for i in I[-1:]) <= Wc[j])

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# Displaying the result
model.display()

ImportError: cannot import name 'range' from 'pyomo.environ' (/usr/local/lib/python3.10/dist-packages/pyomo/environ/__init__.py)

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [16]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Sets
products = ['P1', 'P2', 'P3']
operations = ['smelting', 'mechanisation', 'packaging']

# Decision variables
model.X = Var(products, within=NonNegativeReals)  # Total units produced
model.X_S = Var(products, within=NonNegativeReals)  # Units smelted at PRODA
model.X_sub = Var(['P1', 'P2'], within=NonNegativeReals)  # Subcontracted smelting units

# Parameters
Cp = {
    'P1': {'smelting': 0.30, 'mechanisation': 0.20, 'packaging': 0.30},
    'P2': {'smelting': 0.50, 'mechanisation': 0.10, 'packaging': 0.20},
    'P3': {'smelting': 0.40, 'mechanisation': 0.27, 'packaging': 0.20}
}

Sp = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}

Rp = {
    'P1': {'smelting': 6, 'mechanisation': 6, 'packaging': 3},
    'P2': {'smelting': 10, 'mechanisation': 3, 'packaging': 2},
    'P3': {'smelting': 8, 'mechanisation': 8, 'packaging': 2}
}

Wc = {'smelting': 8000, 'mechanisation': 12000, 'packaging': 10000}

# Objective function
def profit_rule(model):
    return sum(
        Sp[i] * model.X[i]
        - Cp[i]['smelting'] * model.X_S[i]
        - (Cp[i]['subcontracted'] * model.X_sub[i] if i in model.X_sub else 0)
        - Cp[i]['mechanisation'] * model.X[i]
        - Cp[i]['packaging'] * model.X[i]
        for i in products
    )

model.profit = Objective(rule=profit_rule, sense=maximize)

# Constraints

model.constraints = ConstraintList()
for i in products[:-1]:
    model.constraints.add(model.X[i] == model.X_S[i] + model.X_sub[i])

for j in operations:
    model.constraints.add(sum(model.X_S[i]*Rp[(i,j)] for i in products[:-1]) <= Wc[j])
    model.constraints.add(sum(model.X[i]*Rp[(i,j)] for i in products[-1:]) <= Wc[j])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print results
print(f"P1: {model.X['P1']()}, P1_S: {model.X_S['P1']()}, P1_sub: {model.X_sub['P1']()}")
print(f"P2: {model.X['P2']()}, P2_S: {model.X_S['P2']()}, P2_sub: {model.X_sub['P2']()}")
print(f"P3: {model.X['P3']()}")
print(model.profit())


ERROR:pyomo.core:Rule failed when generating expression for Objective profit with index None:
KeyError: 'subcontracted'
ERROR:pyomo.core:Constructing component 'profit' from data=None failed:
    KeyError: 'subcontracted'


KeyError: 'subcontracted'